<a href="https://colab.research.google.com/github/BRAHIMLOUARDI/movie-web-app/blob/main/Welcome_To_Colaboratory1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q keras

In [2]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,Sequential
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

# Path to translation filekk
path_to_data = 'fra.txt'

# Read file
translation_file = open(path_to_data,"r", encoding='utf-8') 
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
pairs = [sentence.split('\t') for sentence in  raw_data]
pairs=pairs[0:192000]



In [ ]:
x=0
for index,pair in enumerate(pairs):
  print(pair[1])
  x=index
  print(x)

print(pairs[58270])


In [ ]:
!unzip  fra-eng.zip

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


In [ ]:
def smart_find(haystack, needle):
    if haystack.startswith(needle+" "):
        return True
    if haystack.endswith(" "+needle):
        return True
    if haystack.find(" "+needle+" ") != -1:
        return True
    return False

In [3]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = "?" + "¡" + '¿'+"."+"!" ;
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

In [ ]:
from Book1 import words 
raw_data1=raw_data[0:60000]
raw_data2=[]
cout=0
for index,raw in enumerate(raw_data1):
  word_in_raw=False
  for word in words:
    clean_raw=clean_sentence(raw)
    if smart_find(clean_raw,word['English']) and not smart_find(clean_raw,word['French']):
      print(index)
      print('\n')
      print(word)
      print('\n')
      print(clean_raw)
      print('\n')
      print('---------')
      
      word_in_raw=True
  if not word_in_raw :
    cout=cout+1
    raw_data2.append(raw)




In [ ]:
raw_data3=raw_data[0:len(raw_data)]

# for word in words:
#   z=word['English']+'\t'+word['French']
#   raw_data3.append(z)





In [ ]:
for word in words:
  z=word['English']+'\t'+word['French']
  raw_data3.append(z)

In [ ]:
# from Book1 import words 
# words2=words
# x=words1[1]['English']
# print(x.split('\/'))
for word in words1:
  word['English']=word['English'][0]
  word['French']=word['French'][0]
  word['Arabic']=word['Arabic'][0]


In [ ]:
for index, word in enumerate(words1):
   if 'Hardy-Weinberg principle'==word['English']:
       print(index)
       print(word)

{'name': 'Adam', 'learning_rate': 0.001, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}
{'name': 'Adam', 'learning_rate': 0.001, 'decay': 0.0, 'beta_1': 0.9, 'beta_2': 0.999, 'epsilon': 1e-07, 'amsgrad': False}


In [ ]:
with open('fra_eng_sc.txt', 'w') as f:
    for raw in raw_data3:
        f.write("%s\n" % raw)

In [ ]:
print(len(words))

1445


In [5]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [6]:

english_sentences = [clean_sentence(pair[0]).replace("\u202f"," ").replace("\xa0"," ").replace("\u200b"," ").replace("«\u2009"," ").replace("\u2009"," ").rstrip().lstrip() for pair in pairs]
french_sentences = [clean_sentence(pair[1]).replace("\u202f"," ").replace("\xa0"," ").replace("\u200b"," ").replace("«\u2009"," ").replace("\u2009"," ").rstrip().lstrip() for pair in pairs]

# Tokenize words
fra_text_tokenized, fra_text_tokenizer = tokenize(french_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length french sentence: {}'.format(len(max(fra_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,key=len))))


# Check language length
french_vocab = len(contant1=str(french_sentences)
file=open("/french_sentences.txt","w+")
file.write(contant1)
file.close()) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("french vocabulary is of {} unique words".format(french_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))

Maximum length french sentence: 23
Maximum length english sentence: 17
french vocabulary is of 28264 unique words
English vocabulary is of 15182 unique words


In [22]:
contant1=str(fra_text_tokenizer.word_index)
file=open("fra_text_tokenizer.txt","w+")
file.write(contant1)
file.close()

In [ ]:
# contant1=str(french_sentences)
# file=open("/french_sentences.txt","w+")
# file.write(contant1)
# file.close()
contant1=str(eng_text_tokenizer.word_index)
file=open("eng_text_tokenizer.txt","w+")
file.write(contant1)
file.close()

In [7]:
max_french_len = int(len(max(fra_text_tokenized,key=len)))
max_english_len = int(len(max(eng_text_tokenized,key=len)))

fra_pad_sentence = pad_sequences(fra_text_tokenized,max_french_len, padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized,max_english_len, padding = "post")




fra_pad_sentence = fra_pad_sentence.reshape(*fra_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

In [65]:
input_sequence = Input(shape=(max_french_len,))
embedding = Embedding(input_dim=french_vocab, output_dim=256,)(input_sequence)

In [66]:
encoder = LSTM(256, return_sequences=False)(embedding)
r_vec = RepeatVector(max_english_len)(encoder)
decoder = LSTM(256, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(english_vocab))(decoder)


In [67]:
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

Model: "model_25"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       [(None, 23)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 23, 256)           7235584   
                                                                 
 lstm_2 (LSTM)               (None, 256)               525312    
                                                                 
 repeat_vector_1 (RepeatVect  (None, 17, 256)          0         
 or)                                                             
                                                                 
 lstm_3 (LSTM)               (None, 17, 256)           525312    
                                                                 
 time_distributed_1 (TimeDis  (None, 17, 15182)        3901774   
 tributed)                                                

In [ ]:
BATCH_SIZE = 32
STEPS_PER_EPOCH = len(fra_pad_sentence) / BATCH_SIZE
SAVE_PERIOD = 10


from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("model/weights1.{epoch:02d}.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', save_freq=int(SAVE_PERIOD * STEPS_PER_EPOCH))

In [ ]:
print(int(SAVE_PERIOD * STEPS_PER_EPOCH))

20000


In [ ]:
model_results = enc_dec_model.fit(fra_pad_sentence, eng_pad_sentence, batch_size=88, epochs=30)


Epoch 1/30
 590/2182 [=======>......................] - ETA: 1:50 - loss: 2.6442 - accuracy: 0.6574

In [ ]:
enc_dec_model.save('my_model_wholefile_exp.h5')



# from google.colab import files
# files.download('my_model_wholefile_exp.h5')

In [62]:
import tensorflow as tf
import keras as keras
model = tf.keras.models.load_model('my_model_wholefile_exp.h5')
# model_results = model.fit(fra_pad_sentence, eng_pad_sentence, batch_size=88, epochs=10)

 # get weights
# modelWeights = model.get_weights()
#  # get optimizer state as it was on last epoch
# modelOptimizer = model.optimizer

#  # ============ Compile Model ============
#  # redefine architecture (newModel=models.Sequential(), etc.)
# newModel=enc_dec_model = Model(input_sequence, Activation('softmax')(logits))

#  # compile
# newModel.compile(optimizer=modelOptimizer,
#                   loss=sparse_categorical_crossentropy,
#                   metrics=['accuracy'])
#  # set trained weights
# newModel.set_weights(modelWeights)

#  # ============ Resume Training ============
# model_results1 = model.fit(fra_pad_sentence, eng_pad_sentence, batch_size=88, epochs=10)

In [ ]:
# import tensorflow as tf
# new_enc_dec_model = tf.keras.models.load_model('my_model.h5')



In [60]:
x=["La nature est pleine de mystère"]
y=fra_text_tokenizer.texts_to_sequences(x)
z=pad_sequences(y, max_french_len, padding = "post")

# print(y)
z=np.reshape(z,(1,max_french_len,1))

def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

index = 8000
# print("The english sentence is: {}".format(english_sentences[index]))
print("The french sentence is: {}".format(x[0]))
print('The predicted sentence is :')
print(logits_to_sentence(model.predict(z)[0], eng_text_tokenizer))


The french sentence is: La nature est pleine de mystère
The predicted sentence is :
nature is full of mystery            


In [64]:
model_results = enc_dec_model.fit(fra_pad_sentence, eng_pad_sentence, batch_size=88, epochs=100)


Epoch 1/100
3133/6000 [==============>...............] - ETA: 1:38 - loss: 2.3302 - accuracy: 0.6670

KeyboardInterrupt: ignored

In [ ]:
 # Lower case the sentence
lower_case_sent = "N'est-ce pas agaçan¡t!.".lower()
print(lower_case_sent)
# Strip punctuation
string_punctuation = "?" + "¡" + '¿'+"."+"!" ;
print(string_punctuation)

clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
print(clean_sentence)
print(string.punctuation)

exp

In [ ]:
#exp

input_sequence1 = Input(shape=(12,))
embedding1 = Embedding(input_dim=7188, output_dim=128,)(input_sequence1)

model1 = Model(inputs=input_sequence1 , outputs=embedding1, name="mnist_model")
model1.compile('rmsprop', 'mse')


input_data = np.array([[1,2,4,5,18,0,0,0,4,10,12,13]])

output_array = model1.predict(input_data)
print(output_array[0][0])


In [ ]:
# for layer in model1.layers: print(layer.get_config(), layer.get_weights())
#print(model1.layers[1].get_weights()[0].shape)

(7188, 128)


In [ ]:
#exp
lstm = LSTM(64, return_sequences=False)(embedding1)

lstmrepeatvector = RepeatVector(6)(lstm)
lstmdecoder = LSTM(64, return_sequences=True, dropout=0.2)(lstmrepeatvector)
lstmlogits = TimeDistributed(Dense(3733))(lstmdecoder)
modellstm = Model(inputs=input_sequence1 , outputs=lstmlogits, name="mnist_model")

In [ ]:
#exp
modellstm.compile('rmsprop', 'mse')


In [ ]:
modellstm.save('my_model_v21.h5')
!pip install tensorflowjs
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(modellstm, 'model')
!zip -r model.zip model

In [ ]:
#EXP
output_array = modellstm.predict(input_data)
reshape_out=np.reshape(output_array[0][0],(3733,-1))
print(reshape_out.shape)


(3733, 1)


In [ ]:
# list_of_lists = [[1,2,3],[4,5,6],[7,8,9]]

# import csv

# with open("out.csv", "w") as f:
#     wr = csv.writer(f)
   
#     wr.writerows(exppairs)

exppairs=pairs[0:10]
file = open("sample.py", "w+")

# Saving the array in a text file
for pair in exppairs :
   file.write(str(pair))
file.close()


In [ ]:
# modelexp = tf.sequential();
# modelexp.add(tf.layers.dense({ units: 1, inputShape: [1] }));

In [ ]:
import tensorflow as tf
import keras as keras
modelexp = Sequential()
# modelexp.add(Input(shape=(1,)))
modelexp.add(Dense(1,input_dim=1))

modelexp.compile(optimizer='sgd', loss=tf.keras.losses.MeanSquaredError( name="mean_squared_error"),metrics=['accuracy'])
xs = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]);
xs=np.reshape(xs,(12,1))
ys = np.array([1, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]);
ys=np.reshape(ys,(12,1))
modelexp.fit(xs, ys,epochs=3000, batch_size=12 )
x1=np.array([40])
x1=np.reshape(x1,(1,1))
print(x1.shape);
print(modelexp.predict(x1))

In [ ]:
modelexp.save('my_model_v21.h5')
!pip install tensorflowjs
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(modelexp, 'model1')
!zip -r model.zip model

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import tensorflow as tf
import keras as keras
modelexp = Sequential()
# modelexp.add(Input(shape=(1,)))
modelexp.add(Dense(1,input_dim=1))

modelexp.compile(optimizer='sgd', loss=tf.keras.losses.MeanSquaredError( name="mean_squared_error"),metrics=['accuracy'])
xs = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]);
xs=np.reshape(xs,(12,1))
ys = np.array([1, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]);
ys=np.reshape(ys,(12,1))
modelexp.fit(xs, ys,epochs=300, batch_size=1 )
x1=np.array([40])
x1=np.reshape(x1,(1,1))
print(x1.shape);
print(modelexp.predict(x1))